<a href="https://colab.research.google.com/github/nzheng32/hw3/blob/main/CXproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CX 4240 Project - University ranking


<img src="https://media.istockphoto.com/photos/law-quadrangle-university-of-michigan-ann-arbor-aerial-view-picture-id1319991057?b=1&k=20&m=1319991057&s=170667a&w=0&h=MXEuQhq56vOInXq7NqZcBynvLxynnBIN8bwgm4dWE-U=" width=800 height=400 />

In [ ]:
# Code to read csv file into colaboratory:
# add shortcut to google drive from the shared folder, then the following code will ask permission to access your google drive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
cd gdrive/MyDrive//Final CX4240

## Imports

We are only using numpy, pandas, and a few plotting functions here. 

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Data Preprocessing

In [ ]:
# Code for Pearson Correlation Matrix

university = pd.read_csv("PCMRank.csv")
# print(university)
corr_matrix = university.corr()
fig, ax = plt.subplots(figsize=(15,15))
sns.heatmap(corr_matrix, annot = True, linewidths = 0.001, ax = ax)
plt.show()

##Candidate Model 1: Linear Regression

In [ ]:
# Code to run Linear Regression on all features in the Pearson Correlation Matrix


x = university.drop('Rankings', axis = 1)
y = university["Rankings"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

x_train_new = x_train.drop('Name', axis = 1)
x_test_new = x_test.drop('Name', axis = 1)


LR = LinearRegression()

LR.fit(x_train_new,y_train)

y_prediction =  LR.predict(x_test_new)

print(pd.concat([pd.Series(np.array(x_test["Name"])), pd.Series(y_prediction)], axis = 1))
# print(list(zip(list(x_test_new.columns), LR.coef_)))

# Accuracy

LR.fit(x_train_new,y_train)
r2_score = LR.score(x_test_new,y_test)
print("Accuracy:", r2_score*100,'%')

# Mean Squared Loss

loss = mse = (np.square(y_prediction - y_test)).mean(axis=None)
print("Loss:", loss)


In [ ]:
# Code to run Linear Regression on only the most correlated features (above |0.6| in Pearson Correlation Matrix)

x = university[['Name', 'SAT Critical Reading 25th percentile score', 'SAT Critical Reading 75th percentile score', 'SAT MATH 25 percentiles score', 'SAT Math 75th percentile score', 'Percent admitted - total', 'Graduation rate - Bachelor degree within 6 years, total']]
y = university["Rankings"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

x_train_new = x_train.drop('Name', axis = 1)
x_test_new = x_test.drop('Name', axis = 1)


LR = LinearRegression()

LR.fit(x_train_new,y_train)

y_prediction =  LR.predict(x_test_new)

print(pd.concat([pd.Series(np.array(x_test["Name"])), pd.Series(y_prediction)], axis = 1))
# print(list(zip(list(x_test_new.columns), LR.coef_)))

# Accuracy

LR.fit(x_train_new,y_train)
r2_score = LR.score(x_test_new,y_test)
print("Accuracy:", r2_score*100,'%')
# Mean Squared Loss

loss = mse = (np.square(y_prediction - y_test)).mean(axis=None)
print("Loss:", loss)


NameError: ignored

##Candidate Model 2: Random Forest Regression

In [ ]:
x = university.drop('Rankings', axis = 1)
y = university["Rankings"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

x_train_new = x_train.drop('Name', axis = 1)
x_test_new = x_test.drop('Name', axis = 1)
accurracy_list = []
mls_loss_list = []
numberTree = []
best_acc = 0
best_loss = 10e3
best_tree = 0

for i in range(80, 200, 5):
  regr = RandomForestRegressor(n_estimators = i, max_depth=16, random_state=3)
  regr.fit(x_train_new, y_train)

  y_prediction =  regr.predict(x_test_new)

  # Accuracy 
  regr.fit(x_train_new,y_train)
  r2_score = regr.score(x_test_new,y_test)


  # Mean Squared Loss
  loss = mse = (np.square(y_prediction - y_test)).mean(axis=None)
  numberTree.append(i)
  accurracy_list.append(r2_score*100)
  mls_loss_list.append(loss)
  if r2_score*100 > best_acc:
    best_acc = r2_score*100
    best_tree = i
  if loss < best_loss:
    best_loss = loss


plt.plot(numberTree, accurracy_list, color='green')
plt.title("Accuracy vs Number of trees used")
plt.xlabel('number of trees') 
plt.ylabel('accuracy') 
plt.show()

plt.plot(numberTree, mls_loss_list, color='green')
plt.title("MLS loss vs Number of trees used")
plt.xlabel('number of trees') 
plt.ylabel('loss') 
plt.show()

print("The best accuracy achieved is " + str(best_acc) + " with number of trees equal to " + str(best_tree))
print("The best loss achieved is " + str(best_loss))


##K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import metrics
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

data = pd.read_csv("College_Fit_Data.csv")

new_data = data.drop(['Name'], axis = 1)



kmeans = KMeans(n_clusters = 44)

y = kmeans.fit_predict(new_data)

data['Cluster'] = y

print(data[['Name', 'Cluster']])

name_cluster = list(zip(data.Name, data.Cluster))

name_cluster_dict = {}
for i in name_cluster:
  if i[1] not in name_cluster_dict:
    a_list = []
    a_list.append(i[0])
    name_cluster_dict[i[1]] = a_list
  else:
    current = name_cluster_dict[i[1]]
    current.append(i[0])
    name_cluster_dict[i[1]] = current

print(name_cluster_dict)





# sil = []
# kmax = 100

# for k in range(2, kmax+1):
#   kmeans = KMeans(n_clusters = k).fit(new_data)
#   labels = kmeans.labels_
#   sil.append(silhouette_score(new_data, labels, metric = 'euclidean'))

# range_of_values = np.array(list(range(1, 100)))
# sil = np.array(sil)
# plt.plot(range_of_values, sil)
# plt.show()
# print(np.amax(sil))

# range_of_values = pd.Series(range_of_values)
# sil = pd.Series(sil)

# sil_values = pd.concat([range_of_values, sil], axis=1)
# sil_values.columns = ['k', 's_coef']

# print(sil_values[sil_values.s_coef == sil_values.s_coef.max()])
